# BRAT Gold Label Adjudication

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(0,'../../ehr-rwe/')

from rwe.contrib.brat import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from shutil import copyfile

infile = "/users/fries/desktop/annotations.txt"
anns = open(infile,'r').read().splitlines()
anns = {name.replace(".ann","") for name in anns}

for annotator in ['jason', 'alison']:
    print(annotator)
    root_fpath = '/Users/fries/Desktop/npj-dev-surveillance-MIMIC-III/v2/'
    filelist = glob.glob(f'{root_fpath}{annotator}/*')
    outdir = f'/Users/fries/Desktop/npj-subset/v2/{annotator}/'
    for fpath in filelist:
        fname_ext = fpath.split("/")[-1]
        fname = fname_ext.split(".")[0]
        if fname in anns:
            copyfile(fpath, f'{outdir}{fname_ext}')



jason
alison


In [53]:
#inputdir = "/Users/fries/Desktop/npj-dev-surveillance-MIMIC-III/"
inputdir = "/Users/fries/Desktop/npj-subset/v2/"

gold = BratAnnotations(inputdir)
gold.annotator_summary()
gold.annotator_agreement(ignore_types=['Concept'], relations_only=True)

ANNOTATOR SUMMARY
-------------------------
jason
-------------------------
ENTITY              962
 - Concept          962
                         
RELATION            182
 - X-at             182
                         
-------------------------
alison
-------------------------
ENTITY              961
 - Concept          961
                         
RELATION            179
 - X-at             179
                         
Kappa Agreement (method=randolph)
OVERALL        (n=539): 0.952
X-at           (n=184): 0.924


In [54]:
import collections

M = gold.aggregate_raters(ignore_types=['Concept'], relations_only=True)

adjudicate = collections.defaultdict(list)

for anno in M:
    if M[anno][0] == 1 and anno.type != 'Concept':
        #print(anno.doc_name, anno, M[anno])
        adjudicate[anno.doc_name].append(anno)

for doc_name in adjudicate:
    print(doc_name)
    for anno in adjudicate[doc_name]:
        print(anno)
    print('-' * 50)

19430_7400_190280
X-at( Concept("pain":433-437), Concept("right chest":405-416) )
X-at( Concept("right chest":405-416), Concept("pain":433-437) )
--------------------------------------------------
25018_7921_177363
X-at( Concept("dislocated":5121-5131), Concept("hemiarthroplasty":5175-5191) )
--------------------------------------------------
2604_68251_150426
X-at( Concept("Removal":499-506), Concept("hardware":510-518) )
--------------------------------------------------
25035_51203_193364
X-at( Concept("hardware":17867-17875), Concept("removal":17876-17883) )
X-at( Concept("removal":10208-10215), Concept("LLE harware":10196-10207) )
X-at( Concept("LLE harware":10196-10207), Concept("removal":10208-10215) )
--------------------------------------------------


In [102]:
# class_map = lambda c: 1 if c["Finding"].attribute("PresentPositive") else 0
# gold.init_labels(class_map, ['Complication'], verbose=True)

48 annotated documents
- 48 2x annotated
Class 0:  120
Class 1:  127
---------------
TOTAL:    247


In [63]:
M = gold.aggregate_raters(ignore_types=['Concept'], relations_only=True)

pain_terms = {
    'pain','tenderness','discomfort','tender', 'painful'
}

n = 0
relations = [x for x in M if x.type == 'X-at']
print(len(relations))
for x in relations:
    if pain_terms.intersection([a.text for a in x.args]):
        n += 1
print(n)

184
147
